<h1 style = "text-align:center"> Chatbot for Pirate Recruitment </h1>
<img src = "https://cdn-images-1.medium.com/max/800/1*SAHhdvu3v9l81OjsfBa_Yw.jpeg" />

Going Merry is a chatbot that helps in recruitment of pirates all around the world. this answer user's simple questions regarding the recruitment process, pre-requisites, etc.

<h2>Libraries</h2>
The main libraries that are used are tensorflow2, keras , numpy and pandas

In [64]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

<h2> Importing the Data </h2>

The data is stored in a json file, which can be imported and used as a pandas dataframe.
This data was manually created by me. hence, it's not that big.<br>
we all know that deep learning requires large chunks of data. but, That is not the case here. I have utilized a neural network architecture powerful enough to handle this small amount of data

In [65]:
#importing the dataset
with open('content.json') as content:
  data1 = json.load(content)

In [66]:
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [67]:
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})

In [68]:
#printing the data
data

,inputs,tags
0,hello,greeting
1,hi there,greeting
2,nice to meet you,greeting
3,hi,greeting
4,hey there,greeting
5,hey,greeting
6,hi there,greeting
7,"hi, nice to meet you",greeting
8,hello there,greeting
9,Thank you,goodbye


In [69]:
data = data.sample(frac=1)

<h2> Pre-Processing </h2>

Important pre-processing such as removing the punctuations, converting to lowercase, encoding the textual data to numerical data are done

In [70]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
data

,inputs,tags
14,goodbye,goodbye
3,hi,greeting
28,nine,guests
20,one,guests
37,located,location
25,six,guests
51,spicy,spicy
6,hi there,greeting
18,table for,reservations
0,hello,greeting


In [71]:
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

Tensorflow's tokenizer assigns a unique token to each distinct word. and padding is done to get all the data to the same length so as to send it to an rnn layer

In [72]:
input_shape = x_train.shape[1]
print(input_shape)

5


In [73]:
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
output_length = le.classes_.shape[0]
print("output length: ",output_length)

number of unique words :  52
output length:  10


<h2>Neural Network</h2>

The Network consist of an embedding layer which is one of the most powerful things in the field of natural language processing. the outputs of the embedding layer is the input of the reccurent layer with lstm gate. then, the output is flattened and a regular dense layer is used with a softmax activation function.

The main part is the embedding layer which gives has a corresponding vector for each word in the dataset

In [74]:
#creating the model

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)

In [75]:
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])

In [76]:
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
2/2 [==============================] - 1s 4ms/step - loss: 2.2976 - accuracy: 0.1852
Epoch 2/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2936 - accuracy: 0.2778
Epoch 3/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2899 - accuracy: 0.3333
Epoch 4/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2858 - accuracy: 0.3704
Epoch 5/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2817 - accuracy: 0.3519
Epoch 6/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2774 - accuracy: 0.2778
Epoch 7/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2727 - accuracy: 0.2593
Epoch 8/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2681 - accuracy: 0.2593
Epoch 9/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2627 - accuracy: 0.2593
Epoch 10/200
2/2 [==============================] - 0s 4ms/step - loss: 2.2571 - accuracy: 0.2593
Epoch 11/200
2/2 [===========

<h2> Model Analysis </h2>

The model got a perfect accuracy of 100%. 

In [77]:
#plotting model accuracy
plt.plot(train.history['accuracy'],label='training set accuracy')
plt.plot(train.history['loss'],label='training set loss')
plt.legend()

<h2> Testing </h2>

I have also tested the model in a way to mimic a human interacting with a bot and got positive results. however, This was very simple because of the tiny amount of data that I have created. but, the same model can also be used with large amount of data

In [ ]:
#chatting
import random


while True:
  texts_p = []
  prediction_input = input('You : ')

  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)

  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()

  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going Merry : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You :  located'


Going Merry :  We are located at 9 Engineering Drive 1, #07-26 EA, Singapore 117575


You :  located


Going Merry :  We are located at 9 Engineering Drive 1, #07-26 EA, Singapore 117575


<h2> Conclusion </h2>
So, This is the Chatbot that I have created with tensorflow2 utilizing the power of embedding matrix.<br>
This was created in memory og <b> Going Merry </b> , a ship from a manga called "One Piece" which revolves around the story of 5 pirates. 